In [6]:
import sys
sys.path.insert(0,'../code/')
import Top
import numpy as np
import random
import time
import pickle
import matplotlib.pyplot as plt
random.seed(12345)
np.random.seed(12345)

## Goal
Hope to use Top algorithm to find the structure in GTEX data set. (Hopefully K is around 10 so it is comparable to other methods)

## Load Data

In [8]:
with open("../data/cis_gene_expression.txt") as f:
    ncols = len(f.readline().split(' '))
data = np.loadtxt("../data/cis_gene_expression.txt",delimiter=' ', skiprows=1,
                 usecols=range(2,ncols+1), dtype = float)

In [9]:
data.shape

(16069, 8555)

In [10]:
sample_id = np.loadtxt("../data/samples_id.txt", delimiter=' ',skiprows=1,
                      usecols=2, dtype = str)
sample_id.shape

(8555,)

In [11]:
gene_id = np.loadtxt("../data/cis_gene_expression.txt",delimiter=' ', skiprows=1,
                 usecols=1, dtype = str)

In [12]:
gene_id.shape

(16069,)

## Run Top using subset of data

In [21]:
p_t = 3000
n_t = 30
N = p_t
rows_t = random.sample(range(1,data.shape[0]),p_t)
data_t = data[rows_t,:n_t]

In [22]:
data_t.shape

(3000, 30)

In [23]:
import time
start = time.time()
top_t = Top.Top(data_t, T = 1)
print("run finished after: ", str(time.time() - start))

run finished after:  2.226332426071167


In [24]:
top_t["K"]

41

## Comment:
K is larger than the number of samples

## Run with total data

In [26]:
## as it takes too long to run, I set T = 1 (default is 10) (T:T: averaged times for selecting L and estimating A )
import time
start = time.time()
top = Top.Top(data, T =1)
print("run finished after: ", str(time.time() - start))

run finished after:  1206.222419977188


In [27]:
top["K"]

348

In [28]:
len(top["Anchor words"])

348

In [124]:
len(top["Anchor groups"])

390

In [125]:
import pickle
with open("top_total","wb") as f:
    pickle.dump(top,f)

### Comment:
So Top uses 348 anchor words, and almost each word decides a group. It seems too large for our purpose.<br>
Below is the corresponding gene id of those anchor words.

In [85]:
#gene_id[top["Anchor words"]]

## Select data with appropriate K (around 10)

In [76]:
np.unique(sample_id)

array(['""', '"Adipose', '"Adrenal', '"Bladder"', '"Blood', '"Blood"',
       '"Brain"', '"Breast"', '"Cervix', '"Colon"', '"Esophagus"',
       '"Fallopian', '"Heart"', '"Kidney"', '"Liver"', '"Lung"',
       '"Muscle"', '"Nerve"', '"Ovary"', '"Pancreas"', '"Pituitary"',
       '"Prostate"', '"Salivary', '"Skin"', '"Small', '"Spleen"',
       '"Stomach"', '"Testis"', '"Thyroid"', '"Uterus"', '"Vagina"'],
      dtype='<U11')

In [77]:
Brain_indx = np.where(sample_id == '"Brain"')[0]

In [13]:
data_brain = data[:,Brain_indx]

In [14]:
data_brain.shape

(16069, 1259)

In [15]:
start = time.time()
top_brain = Top.Top(data_brain, T=1)
print("run finished after: ", str(time.time() - start))

run finished after:  3345.3046033382416


In [16]:
top_brain["K"]

390

### Comment: 
Get the same K from Brain as using all samples. Below we will see the anchor words are the same

In [20]:
import pickle
with open("top_brain","wb") as f:
    pickle.dump(top_brain,f)

In [2]:
import pickle
with open("top_total","rb") as t:
    top = pickle.load(t)
    
with open("top_brain","rb") as t:
    top_brain = pickle.load(t)

In [19]:
top["Anchor words"].sort() == top_brain["Anchor groups"].sort()

True

## Try on even smaller samples, like brain


In [13]:
heart_indx = np.where(sample_id == '"Heart"')[0]
data_heart = data[:,heart_indx]

In [14]:
data_heart.shape

(16069, 412)

In [15]:
## select only a small proportiion of genes
data_heart_small = data_heart[:3000,:]

In [16]:
start = time.time()
top_heart_small = Top.Top(data_heart_small, T = 1)
print("run finished after: ", str(time.time()-start))

run finished after:  26.968742847442627


In [17]:
top_heart_small["K"]

139

## Comment:
K is still much larger than what we want